In [1]:
# #Donwload the datasets
!rm -rf data/
!mkdir data/

!curl https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/orientme/v0.2/training.tar.gz -o data/training.tar.gz
!curl https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/orientme/v0.2/test.tar.gz -o data/test.tar.gz
!curl https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/orientme/v0.2/sample_submission.csv -o data/sample_submission.csv
!tar xvzf data/training.tar.gz -C data/
!tar xvzf data/test.tar.gz -C data/

Streaming output truncated to the last 5000 lines.
images/008254.jpg
images/007167.jpg
images/008532.jpg
images/007601.jpg
images/005016.jpg
images/005002.jpg
images/008526.jpg
images/007615.jpg
images/009638.jpg
images/008240.jpg
images/007173.jpg
images/005764.jpg
images/006245.jpg
images/009176.jpg
images/008268.jpg
images/005994.jpg
images/006523.jpg
images/009610.jpg
images/007629.jpg
images/006537.jpg
images/009604.jpg
images/005980.jpg
images/005758.jpg
images/006251.jpg
images/009162.jpg
images/008297.jpg
images/009189.jpg
images/009837.jpg
images/009823.jpg
images/008283.jpg
images/005957.jpg
images/007198.jpg
images/006286.jpg
images/007826.jpg
images/007832.jpg
images/006292.jpg
images/005943.jpg
images/005214.jpg
images/007403.jpg
images/008730.jpg
images/007365.jpg
images/008056.jpg
images/009348.jpg
images/005572.jpg
images/005566.jpg
images/007371.jpg
images/008042.jpg
images/006709.jpg
images/007417.jpg
images/008724.jpg
images/005200.jpg
images/009412.jpg
images/006721

In [0]:
## Now the data is available at the following locations:

TRAINING_IMAGES_FOLDER = "data/training/images/"
TRAINING_LABELS_PATH = "data/training/labels.csv"
TEST_IMAGES_FOLDER = "data/images"
SAMPLE_SUBMISSION_FILE_PATH = "data/sample_submission.csv"


## Import packages

In [0]:
import os
import tqdm

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image 


## Load Data
We use PIL library to load our images. Here we are creating our array where our input features are the mean colours and output features are the rotations along the x axis.

In [4]:
training_labels_df = pd.read_csv(TRAINING_LABELS_PATH)

FP = []
rot = []
for _idx, row in tqdm.tqdm(training_labels_df.iterrows(), total=training_labels_df.shape[0]):
    filepath = os.path.join(
        TRAINING_IMAGES_FOLDER,
        row.filename
    )

    FP.append(filepath)
    rot.append(row.xRot)

FP = np.array(FP)
rot = np.array(rot)


100%|██████████| 5000/5000 [00:00<00:00, 8179.16it/s]


In [0]:
import cv2

In [6]:
tr_imgs = []

for i in tqdm.tqdm(FP):
  im = cv2.imread(i)
  im = cv2.resize(im, (32,32))
  tr_imgs.append(im.reshape(1,-1))
tr_imgs = np.array(tr_imgs)

100%|██████████| 5000/5000 [00:13<00:00, 360.14it/s]


In [7]:
import glob

TEST_DATA = []
TEST_FILENAMES = []

for _test_image_path in tqdm.tqdm(glob.glob(os.path.join(TEST_IMAGES_FOLDER, "*.jpg"))):
    filename = os.path.basename(_test_image_path)
    # im = Image.open(_test_image_path)
    
    # data_X = pre_process_data_X(im)
    # TEST_DATA.append(data_X)
    TEST_FILENAMES.append(os.path.join(TEST_IMAGES_FOLDER,filename))

100%|██████████| 5001/5001 [00:00<00:00, 407580.33it/s]


In [0]:
TEST_FILENAMES = sorted(TEST_FILENAMES)

In [9]:
ts_imgs = []

for i in tqdm.tqdm(TEST_FILENAMES):
  im = cv2.imread(i)
  im = cv2.resize(im, (32,32))
  ts_imgs.append(im.reshape(1,-1))
ts_imgs = np.array(ts_imgs)

100%|██████████| 5001/5001 [00:13<00:00, 370.88it/s]


In [0]:
TS = ts_imgs.reshape(5001,-1)

In [0]:
TS = np.array(TS, dtype=np.float32)

In [0]:
TR = tr_imgs.reshape(5000,-1)

In [0]:
TR = np.array(TR, dtype=np.float32)

In [0]:
D = np.dot(TS,TR.T)

In [0]:
A = np.linalg.norm(TR,axis=1).reshape(1,-1)**2 + np.linalg.norm(TS,axis=1).T.reshape(-1,1)**2 - 2*D

In [0]:
B = np.argsort(A, axis=1)

In [0]:
idxx = B[:, 0]

In [0]:
sub1 = rot[idxx]

In [0]:
files = []
for i in TEST_FILENAMES:
  files.append(i.split('/')[-1])

In [0]:
test_df = pd.DataFrame(sub1, columns=['xRot'])
test_df["filename"] = files

In [0]:
test_df.to_csv('submission_orientme.csv', index=False)